In [1]:
import torch
from torch import nn
import random

from input_generator import get_input, __create_one_hot_seq
from model import RNNModel

import os
from datetime import datetime

import numpy as np

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(str(device) + ' will be used')

cuda will be used


In [3]:
max_len_limit = 15 # must be an odd number
n_input = 10000
alphabet = ['0','1','2','3','4','5','6','7','8','9','-','+','/','*',' '] 

input_feature_num = len(alphabet)


In [4]:
model = RNNModel(input_feature_num=input_feature_num, device=device)
# add teh model path to the string if you want to load a model
# model.load_state_dict(torch.load('/home/sadullah/calculator/model_logs/##'))
model.to(device)

RNNModel(
  (rnn): RNN(15, 100)
  (fc): Linear(in_features=100, out_features=1, bias=True)
)

In [5]:
n_epochs = 10000
lr=0.005

loss_fn = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = loss_fn.to(device)

In [ ]:
test_name = "add-subt-div-mul"

input_seq, target_values = get_input(n_input=n_input, max_len_limit=max_len_limit, max_digit=1, alphabet=alphabet)
target_values = target_values.to(device)

bets_model = None
best_loss = 99999999

time = datetime.now()
time = str(time.month) + "-" + str(time.day) + "-" + str(time.hour) + "-" + str(time.minute)

#dir_name = "/home/sadullah/calculator/model_logs/{}_{}".format(test_name, time)
#os.mkdir(dir_name)

for epoch in range(1,n_epochs+1):
    
    if epoch % 1000 == 0:
        input_seq, target_values = get_input(n_input=n_input, max_len_limit=max_len_limit, max_digit=1, alphabet=alphabet)
        target_values = target_values.to(device)
    
    optimizer.zero_grad()
    input_seq = input_seq.to(device)
    output = model(input_seq)
    loss = loss_fn(output, target_values)
    loss.backward()
    optimizer.step()
    
    if epoch%10 == 0:
        
        if loss < best_loss:
            best_loss = loss
            best_model = model
            time = datetime.now()
            time = str(time.month) + "-" + str(time.day) + "-" + str(time.hour) + "-" + str(time.minute)
            #torch.save(model.state_dict(),(dir_name+'/{0}_{1:.4f}.pth').format(epoch,loss))
        
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/10000............. Loss: 35.0940
Epoch: 20/10000............. Loss: 35.0854
Epoch: 30/10000............. Loss: 35.0863
Epoch: 40/10000............. Loss: 35.0864
Epoch: 50/10000............. Loss: 35.0860
Epoch: 60/10000............. Loss: 35.0856
Epoch: 70/10000............. Loss: 35.0854
Epoch: 80/10000............. Loss: 35.0851
Epoch: 90/10000............. Loss: 35.0814
Epoch: 100/10000............. Loss: 34.7592
Epoch: 110/10000............. Loss: 34.1766
Epoch: 120/10000............. Loss: 33.9140
Epoch: 130/10000............. Loss: 33.6812
Epoch: 140/10000............. Loss: 33.5550
Epoch: 150/10000............. Loss: 33.4730
Epoch: 160/10000............. Loss: 33.6724
Epoch: 170/10000............. Loss: 33.4562
Epoch: 180/10000............. Loss: 33.4318
Epoch: 190/10000............. Loss: 33.5025
Epoch: 200/10000............. Loss: 33.1859
Epoch: 210/10000............. Loss: 32.9749
Epoch: 220/10000............. Loss: 33.5153
Epoch: 230/10000............. Loss: 33.68

Epoch: 1860/10000............. Loss: 41.8812
Epoch: 1870/10000............. Loss: 41.9964
Epoch: 1880/10000............. Loss: 41.2279
Epoch: 1890/10000............. Loss: 41.0494
Epoch: 1900/10000............. Loss: 42.9373
Epoch: 1910/10000............. Loss: 41.6838
Epoch: 1920/10000............. Loss: 41.1816
Epoch: 1930/10000............. Loss: 41.2501
Epoch: 1940/10000............. Loss: 40.9350
Epoch: 1950/10000............. Loss: 40.8435
Epoch: 1960/10000............. Loss: 40.9127
Epoch: 1970/10000............. Loss: 41.0871
Epoch: 1980/10000............. Loss: 41.3619
Epoch: 1990/10000............. Loss: 42.2572
Epoch: 2000/10000............. Loss: 21.5260
Epoch: 2010/10000............. Loss: 20.8962
Epoch: 2020/10000............. Loss: 20.8547
Epoch: 2030/10000............. Loss: 20.6179
Epoch: 2040/10000............. Loss: 20.5643
Epoch: 2050/10000............. Loss: 20.6216
Epoch: 2060/10000............. Loss: 20.9693
Epoch: 2070/10000............. Loss: 20.6886
Epoch: 208

Epoch: 3690/10000............. Loss: 26.3342
Epoch: 3700/10000............. Loss: 26.3742
Epoch: 3710/10000............. Loss: 26.7260
Epoch: 3720/10000............. Loss: 26.9485
Epoch: 3730/10000............. Loss: 27.0210
Epoch: 3740/10000............. Loss: 26.4613
Epoch: 3750/10000............. Loss: 26.2565
Epoch: 3760/10000............. Loss: 26.2669
Epoch: 3770/10000............. Loss: 26.4885
Epoch: 3780/10000............. Loss: 26.2819
Epoch: 3790/10000............. Loss: 27.2341
Epoch: 3800/10000............. Loss: 26.6472
Epoch: 3810/10000............. Loss: 26.8001
Epoch: 3820/10000............. Loss: 26.5928
Epoch: 3830/10000............. Loss: 26.4628
Epoch: 3840/10000............. Loss: 26.5930
Epoch: 3850/10000............. Loss: 26.3141
Epoch: 3860/10000............. Loss: 26.3537
Epoch: 3870/10000............. Loss: 26.3102
Epoch: 3880/10000............. Loss: 26.8651
Epoch: 3890/10000............. Loss: 26.1188
Epoch: 3900/10000............. Loss: 26.2356
Epoch: 391

Epoch: 5520/10000............. Loss: 23.3333
Epoch: 5530/10000............. Loss: 23.3934
Epoch: 5540/10000............. Loss: 23.3825
Epoch: 5550/10000............. Loss: 23.3687
Epoch: 5560/10000............. Loss: 23.9796
Epoch: 5570/10000............. Loss: 24.1937
Epoch: 5580/10000............. Loss: 23.6057
Epoch: 5590/10000............. Loss: 23.5102
Epoch: 5600/10000............. Loss: 23.2496
Epoch: 5610/10000............. Loss: 23.4079
Epoch: 5620/10000............. Loss: 23.1949
Epoch: 5630/10000............. Loss: 23.0998
Epoch: 5640/10000............. Loss: 23.5420
Epoch: 5650/10000............. Loss: 23.7801
Epoch: 5660/10000............. Loss: 23.2430
Epoch: 5670/10000............. Loss: 23.7501
Epoch: 5680/10000............. Loss: 23.9290
Epoch: 5690/10000............. Loss: 24.0065
Epoch: 5700/10000............. Loss: 23.7509
Epoch: 5710/10000............. Loss: 23.4910
Epoch: 5720/10000............. Loss: 23.5507
Epoch: 5730/10000............. Loss: 23.6808
Epoch: 574

Epoch: 7350/10000............. Loss: 14.7677
Epoch: 7360/10000............. Loss: 14.3991
Epoch: 7370/10000............. Loss: 14.3557
Epoch: 7380/10000............. Loss: 14.0720
Epoch: 7390/10000............. Loss: 14.6007
Epoch: 7400/10000............. Loss: 14.7486
Epoch: 7410/10000............. Loss: 14.2504
Epoch: 7420/10000............. Loss: 14.2955
Epoch: 7430/10000............. Loss: 14.1079
Epoch: 7440/10000............. Loss: 14.6823
Epoch: 7450/10000............. Loss: 14.4695
Epoch: 7460/10000............. Loss: 14.1115
Epoch: 7470/10000............. Loss: 14.0838
Epoch: 7480/10000............. Loss: 14.2643
Epoch: 7490/10000............. Loss: 14.1578
Epoch: 7500/10000............. Loss: 14.1551
Epoch: 7510/10000............. Loss: 14.1910
Epoch: 7520/10000............. Loss: 14.4232
Epoch: 7530/10000............. Loss: 14.1437
Epoch: 7540/10000............. Loss: 13.9595
Epoch: 7550/10000............. Loss: 14.1201
Epoch: 7560/10000............. Loss: 14.3477
Epoch: 757

In [ ]:
def predict(model, seq):
    while len(seq) < max_len_limit: seq += ' '
    one_hot_input = __create_one_hot_seq(alphabet, max_len_limit, [seq])
    input_seq = torch.FloatTensor(one_hot_input).permute(1,0,2)
    input_seq = input_seq.to(device)
    output = model(input_seq)
    return output

In [ ]:
x = '9*9/9+9'
y = '5/3+5*2'
z = '2*3/5-7'
r = '9/3*5+2'
a = predict(model, x).data
b = predict(model, y).data
c = predict(model, z).data
d = predict(model, r).data

In [ ]:
a,b,c,d